# [빅데이터 5조] 02. 데이터 전처리
### 목차
- [2.1 코로나 확진자 데이터 수 전처리](#2.1-코로나-확진자-데이터-수-전처리)
- [2.2 게임 이용자 수 데이터 전처리](#2.2-게임-이용자-수-데이터-전처리)
- [2.3 기존 리뷰 데이터 전처리](#2.3-기존-리뷰-데이터-전처리)
- [2.4 신규 리뷰 데이터 전처리](#2.4-신규-리뷰-데이터-전처리)
- [2.5 리뷰 데이터 병합, 전처리, 저장](#2.5-리뷰-데이터-병합,-전처리,-저장)

In [1]:
from datetime import datetime
import gc
import os
import re
import pandas as pd

---
### 2.1 코로나 확진자 데이터 수 전처리

In [42]:
# cv is Covid
cv_df = pd.read_csv("./dataset/CONVENIENT_global_confirmed_cases.csv", encoding="utf-8-sig")

display(cv_df)

,Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,...,United Kingdom.11,Uruguay,Uzbekistan,Vanuatu,Venezuela,Vietnam,West Bank and Gaza,Yemen,Zambia,Zimbabwe
0,Province/State,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Australian Capital Territory,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1/23/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
2,1/24/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1/25/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1/26/20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673,11/25/21,112.0,516.0,161.0,140.0,25.0,3.0,2260.0,592.0,8.0,...,46654.0,240.0,219.0,0.0,1654.0,12450.0,257.0,5.0,13.0,27.0
674,11/26/21,27.0,440.0,193.0,146.0,25.0,0.0,1912.0,675.0,7.0,...,49344.0,214.0,206.0,0.0,907.0,13109.0,266.0,2.0,6.0,62.0
675,11/27/21,19.0,405.0,163.0,0.0,9.0,0.0,1521.0,517.0,7.0,...,39567.0,291.0,218.0,0.0,686.0,16067.0,0.0,2.0,20.0,0.0
676,11/28/21,28.0,418.0,172.0,0.0,5.0,0.0,888.0,409.0,7.0,...,36507.0,167.0,232.0,0.0,650.0,12936.0,0.0,6.0,5.0,115.0


In [43]:
# Province/State 행 제거
cv_df = cv_df.drop(cv_df.index[0])

# 열 이름 변경
cv_df = cv_df.rename(columns={"Country/Region": "date"})

# 시계열 데이터 형식 수정 (mm/dd/yy -> yyyy/mm/dd)
def edit_date(date):
    # 한자리수 일/월에 자리수를 나타내는 0 추가
    date = "/".join([x.zfill(2) for x in date.split("/")])
    return datetime.strptime(date, "%m/%d/%y").strftime("%Y-%m-%d")

cv_df["date"] = cv_df.apply(
    lambda x: edit_date(x["date"]), axis=1
)

# date 열을 데이터프레임의 인덱스로 변경
cv_df = cv_df.set_index("date")
cv_df.index.name = None

# 일별 합계 계산
cv_df["total_confirm"] = cv_df.sum(axis=1, numeric_only=True)
cv_df = cv_df[["total_confirm"]]

# 결측값 처리(0) 및 명시적 형변환
cv_df = cv_df.fillna(0)
cv_df = cv_df.astype("int")

# 결측치로 추정되는 데이터 처리()
cv_df.sort_values(by="total_confirm", ascending=False) # 2020-12-10 일자의 데이터가 이상치로 추정됨
cv_df.loc["2020-12-10"] = (cv_df.loc["2020-12-09"]+cv_df.loc["2020-12-11"]) / 2

display(cv_df)
cv_df.to_csv("./dataset/interim_data/covid_sum.csv", encoding="utf-8-sig")

,total_confirm
2020-01-23,3
2020-01-24,9
2020-01-25,7
2020-01-26,10
2020-01-27,6
...,...
2021-11-25,589051
2021-11-26,591520
2021-11-27,409868
2021-11-28,427210


---
### 2.2 게임 이용자 수 데이터 전처리

In [44]:
# ap is Average Players
ap_df = pd.read_csv("./dataset/interim_data/avg_players.csv", encoding="utf-8-sig")
display(ap_df)

,name,month,avg_players
0,Counter-Strike,2021-09,8461.12
1,Counter-Strike,2021-08,8851.19
2,Counter-Strike,2021-07,9137.76
3,Counter-Strike,2021-06,9497.45
4,Counter-Strike,2021-05,10740.95
...,...,...,...
839877,我的侠客 试玩版,2021-09,21.66
839878,The Planet Crafter Demo,2021-09,35.16
839879,Haunted Hotel: The Axiom Butcher Collector's E...,2021-09,3.32
839880,Karryn's Prison Demo,2021-09,3.98


In [45]:
ap_df = ap_df.groupby("month").sum()
ap_df.index.name = None

display(ap_df)
ap_df.to_csv("./dataset/interim_data/avg_players_sum.csv", encoding="utf-8-sig")

,avg_players
2012-07,527483.97
2012-08,499853.58
2012-09,527756.88
2012-10,502165.89
2012-11,585970.59
...,...
2021-05,4579001.33
2021-06,4356320.79
2021-07,4497112.90
2021-08,4628024.88


---
### 2.3 기존 리뷰 데이터 전처리

In [11]:
# https://www.kaggle.com/najzeko/steam-reviews-2021
# er is Existing Review
er_df = pd.read_csv("./dataset/existing_review/steam_reviews.csv", encoding="utf-8", index_col = 0)

,app_id,app_name,review_id,language,review,timestamp_created,timestamp_updated,recommended,votes_helpful,votes_funny,...,steam_purchase,received_for_free,written_during_early_access,author.steamid,author.num_games_owned,author.num_reviews,author.playtime_forever,author.playtime_last_two_weeks,author.playtime_at_review,author.last_played
0,292030,The Witcher 3: Wild Hunt,85185598,schinese,不玩此生遗憾，RPG游戏里的天花板，太吸引人了,1611381629,1611381629,True,0,0,...,True,False,False,76561199095369542,6,2,1909.0,1448.0,1909.0,1.611343e+09
1,292030,The Witcher 3: Wild Hunt,85185250,schinese,拔DIAO无情打桩机--杰洛特!!!,1611381030,1611381030,True,0,0,...,True,False,False,76561198949504115,30,10,2764.0,2743.0,2674.0,1.611386e+09
2,292030,The Witcher 3: Wild Hunt,85185111,schinese,巫师3NB,1611380800,1611380800,True,0,0,...,True,False,False,76561199090098988,5,1,1061.0,1061.0,1060.0,1.611384e+09
3,292030,The Witcher 3: Wild Hunt,85184605,english,"One of the best RPG's of all time, worthy of a...",1611379970,1611379970,True,0,0,...,True,False,False,76561199054755373,5,3,5587.0,3200.0,5524.0,1.611384e+09
4,292030,The Witcher 3: Wild Hunt,85184287,schinese,大作,1611379427,1611379427,True,0,0,...,True,False,False,76561199028326951,7,4,217.0,42.0,217.0,1.610788e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21747371,546560,Half-Life: Alyx,65645125,english,Holy balls,1584986704,1584986704,True,0,0,...,True,False,False,76561197967033932,431,7,432.0,0.0,29.0,1.586628e+09
21747372,546560,Half-Life: Alyx,65645115,english,"A tripod thing bears down on you, it looks imm...",1584986693,1584986693,True,0,0,...,False,False,False,76561198046495482,70,10,3296.0,15.0,24.0,1.611080e+09
21747373,546560,Half-Life: Alyx,65645100,english,Honestly this is the best vr game ever,1584986673,1584986673,True,0,0,...,True,False,False,76561198176379749,79,17,2654.0,0.0,34.0,1.591634e+09
21747374,546560,Half-Life: Alyx,65645066,english,Smooth turning is not working right now.\nIt a...,1584986631,1586382422,True,0,0,...,True,False,False,76561198041763187,140,3,210.0,0.0,12.0,1.589715e+09


In [ ]:
# 리뷰 분석에 사용될 열 선별, recomendded -> 긍정적 / 부정적 여부
er_df = er_df[["app_id", "app_name", "review_id", "language", "review", "recommended", "timestamp_created"]]

# Steam API 문서의 내용에 따라 기존 리뷰 데이터의 열 이름을 신규 리뷰 데이터 기준으로 변경
er_df.columns = ["app_id", "app_name", "recommendationid", "language", "review", "voted_up", "timestamp_created"]

display(er_df)

---
### 2.4 신규 리뷰 데이터 전처리

In [18]:
# nr is New Review
nr_list = [x for x in os.listdir("./dataset/new_review") if ".csv" in x]
nr_df_list = []

for nr in nr_list:
    # 리뷰 데이터 불러오기, DtypeWarning 에러 방지
    nr_df = pd.read_csv(f"./dataset/new_review/{nr}", encoding="utf-8-sig", low_memory=False)
    
    # 리뷰 분석에 사용될 열 선별
    nr_df = nr_df[["app_id", "app_name", "recommendationid", "language", "review", "voted_up", "timestamp_created"]]

    nr_df_list.append(nr_df)
    
nr_df = pd.concat(nr_df_list, axis=0, ignore_index=True)

display(nr_df)

,app_id,app_name,recommendationid,language,review,voted_up,timestamp_created
0,10090,Call of Duty: World at War,103197756,english,Zombos good,True,1637487359
1,10090,Call of Duty: World at War,103196955,english,DO NOT BUY THIS GAME. World at war is a great ...,False,1637486299
2,10090,Call of Duty: World at War,103194172,english,12/10 would mod the og cod zombies experience ...,True,1637482574
3,10090,Call of Duty: World at War,103189070,russian,Отличный шутер про 2 мировую войну.,True,1637475438
4,10090,Call of Duty: World at War,103186807,english,"if u dont play this, u aint black.",True,1637472284
...,...,...,...,...,...,...,...
10460514,99910,Puzzle Pirates,3692155,english,This is not even a game,True,1314818724
10460515,99910,Puzzle Pirates,4498620,english,SHIVER ME TIMBERS! ...Why does steam give me t...,False,1314818654
10460516,99910,Puzzle Pirates,276151,english,Highly recommended blend of MMO and puzzles. T...,True,1314814692
10460517,99910,Puzzle Pirates,546554,english,Yarrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr!,True,1314812971


---
### 2.5 리뷰 데이터 병합, 전처리, 저장

In [20]:
# tr is Total Review
tr_df = pd.concat([er_df, nr_df], axis=0, ignore_index=True)

display(tr_df)

,app_id,app_name,recommendationid,language,review,voted_up,timestamp_created
0,292030,The Witcher 3: Wild Hunt,85185598,schinese,不玩此生遗憾，RPG游戏里的天花板，太吸引人了,True,1611381629
1,292030,The Witcher 3: Wild Hunt,85185250,schinese,拔DIAO无情打桩机--杰洛特!!!,True,1611381030
2,292030,The Witcher 3: Wild Hunt,85185111,schinese,巫师3NB,True,1611380800
3,292030,The Witcher 3: Wild Hunt,85184605,english,"One of the best RPG's of all time, worthy of a...",True,1611379970
4,292030,The Witcher 3: Wild Hunt,85184287,schinese,大作,True,1611379427
...,...,...,...,...,...,...,...
32207885,99910,Puzzle Pirates,3692155,english,This is not even a game,True,1314818724
32207886,99910,Puzzle Pirates,4498620,english,SHIVER ME TIMBERS! ...Why does steam give me t...,False,1314818654
32207887,99910,Puzzle Pirates,276151,english,Highly recommended blend of MMO and puzzles. T...,True,1314814692
32207888,99910,Puzzle Pirates,546554,english,Yarrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr!,True,1314812971


In [5]:
# 중복 리뷰 제거
print(f"중복 리뷰 제거 전 리뷰 개수: {tr_df.shape[0]}")
tr_df.drop_duplicates(["recommendationid"], keep="last") # 
print(f"중복 리뷰 제거 후 리뷰 개수: {tr_df.shape[0]}")

중복 리뷰 제거 전 리뷰 개수: 32207890
중복 리뷰 제거 후 리뷰 개수: 32207890


In [6]:
# 리뷰 내용이 결측치인 행에 대한 1차적 처리
print(f"결측치 제거 전 리뷰 개수: {tr_df.shape[0]}")
tr_df = tr_df.dropna(axis=0, how="any")
print(f"결측치 제거 후 리뷰 개수: {tr_df.shape[0]}")

결측치 제거 전 리뷰 개수: 32207890
결측치 제거 후 리뷰 개수: 32154271


In [7]:
# 같은 언어로 분류되는 포르투갈어 + 브라질리어 병합
tr_df.loc[tr_df["language"] == "brazilian", "language"] = "portuguese"

In [8]:
# 사분면 개념을 활용하여 리뷰 데이터의 속성 추가
pheic_point = int(datetime(2020, 1, 31, 12, 0, 0).timestamp()) # 정확한 선포 시간이 특정되지 않았으므로 중간 시간 사용
print(f"PHEIC 선포 타임 스탬프 수치: {pheic_point}")

PHEIC 선포 타임 스탬프 수치: 1580439600


[사분면 별 의미]
```                                
                         │
 before pheic / positive │ after pheic / positive 
             (2)         │            (1)
 ────────────────────────┼───────────────────────
                         │
 before pheic / negative │ after pheic / negative 
             (3)         │            (4)

                                 
```

In [9]:
# quadrant 열을 추가한 뒤 1 사분면으로 초기화
tr_df["quadrant"] = 1

# 명시적인 형 지정
tr_df = tr_df.astype({"timestamp_created": int, "quadrant": int})

# tr_df.loc[(tr_df["timestamp_created"] >= pheic_point) & (tr_df["voted_up"] == True),  "quadrant"] = 1
tr_df.loc[(tr_df["timestamp_created"] <  pheic_point) & (tr_df["voted_up"] == True),  "quadrant"] = 2
tr_df.loc[(tr_df["timestamp_created"] <  pheic_point) & (tr_df["voted_up"] == False), "quadrant"] = 3
tr_df.loc[(tr_df["timestamp_created"] >= pheic_point) & (tr_df["voted_up"] == False), "quadrant"] = 4

display(tr_df)

,app_id,app_name,recommendationid,language,review,voted_up,timestamp_created,quadrant
0,292030,The Witcher 3: Wild Hunt,85185598,schinese,不玩此生遗憾，RPG游戏里的天花板，太吸引人了,True,1611381629,1
1,292030,The Witcher 3: Wild Hunt,85185250,schinese,拔DIAO无情打桩机--杰洛特!!!,True,1611381030,1
2,292030,The Witcher 3: Wild Hunt,85185111,schinese,巫师3NB,True,1611380800,1
3,292030,The Witcher 3: Wild Hunt,85184605,english,"One of the best RPG's of all time, worthy of a...",True,1611379970,1
4,292030,The Witcher 3: Wild Hunt,85184287,schinese,大作,True,1611379427,1
...,...,...,...,...,...,...,...,...
32207885,99910,Puzzle Pirates,3692155,english,This is not even a game,True,1314818724,2
32207886,99910,Puzzle Pirates,4498620,english,SHIVER ME TIMBERS! ...Why does steam give me t...,False,1314818654,3
32207887,99910,Puzzle Pirates,276151,english,Highly recommended blend of MMO and puzzles. T...,True,1314814692,2
32207888,99910,Puzzle Pirates,546554,english,Yarrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr!,True,1314812971,2


In [ ]:
tr_df.to_csv("./dataset/preprocessed/total_reviews.csv", index=False, encoding="utf-8-sig")